# Model validation

Our goal is to validate the models selected in the model refinement and selection stage. Ultimately, we aim to choose a single final model.

## Imports

In [1]:
# Set module path
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from pathlib import Path
import numpy as np
import pandas as pd
import statsmodels.api as sm
from patsy import dmatrices
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from math import floor

import src.regression as rg
import src.subset_selection as ss

## Data

In [3]:
data_dir = Path("../data/processed/")
data_file = data_dir / "ca-schools.csv"

df = pd.read_csv(data_file, index_col=None)
df.head()

,students,teachers,calworks,lunch,computer,expenditure,income,english,math
0,-2433.792857,-118.167377,-12.735842,-42.664437,-236.383333,1072.503592,7.373412,-15.768155,690.000000
1,-2388.792857,-117.917377,2.170659,3.211462,-202.383333,-213.026682,-5.492588,-11.184822,661.900024
2,-1078.792857,-46.167375,41.786257,31.617364,-134.383333,189.547049,-6.338588,14.231847,650.900024
3,-2385.792857,-115.067376,23.229357,32.343965,-218.383333,1789.423514,-6.338588,-15.768155,643.500000
4,-1293.792857,-57.567376,19.862559,33.721765,-132.383333,-76.419748,-6.236255,-1.910478,639.900024


## Variables and models

$Y$ - math

$X_0$ - intercept

$X_1$ - students

$X_2$ - teachers

$X_3$ - calworks

$X_4$ - lunch

$X_5$ - computer

$X_6$ - expenditure

$X_7$ - income

$X_8$ - english

**Model 1**: $Y = X_0 + X_3 + X_4 + X_5 + X_6 + X_7 + X_8 + X_3 X_7 + X_6 X_7 + \epsilon$

**Model 2**: $Y = X_0 + X_4 + X_7 + X_8 + \epsilon$

**Model 3**: $Y = X_0 + X_4 + X_7 + X_8 + X_4 X_7 + \epsilon$

In [4]:
fm1 = """math ~ calworks + lunch + income + english + calworks:income"""

y1, X1 = dmatrices(fm1, data=df, return_type='dataframe')

mod1 = sm.OLS(y1, X1)
res1 = mod1.fit()

print(res1.summary())


from src.functions import savetable

fig_dir = Path("../reports/figures/")

string = res1.summary().as_latex()
txt_file = open(fig_dir / "35-final-mod1-summary.tex", 'w')
txt_file.write(string)
txt_file.close()


                            OLS Regression Results                            
Dep. Variable:                   math   R-squared:                       0.728
Model:                            OLS   Adj. R-squared:                  0.725
Method:                 Least Squares   F-statistic:                     221.7
Date:                Thu, 28 Jul 2022   Prob (F-statistic):          1.18e-114
Time:                        20:04:22   Log-Likelihood:                -1553.2
No. Observations:                 420   AIC:                             3118.
Df Residuals:                     414   BIC:                             3143.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept         651.9168      0.682    9

In [5]:
fm2 = """math ~ lunch + income + english"""

y2, X2 = dmatrices(fm2, data=df, return_type='dataframe')

mod2 = sm.OLS(y2, X2)
res2 = mod2.fit()

print(res2.summary())

string = res2.summary().as_latex()
txt_file = open(fig_dir / "36-final-mod2-summary.tex", 'w')
txt_file.write(string)
txt_file.close()

                            OLS Regression Results                            
Dep. Variable:                   math   R-squared:                       0.721
Model:                            OLS   Adj. R-squared:                  0.719
Method:                 Least Squares   F-statistic:                     357.5
Date:                Thu, 28 Jul 2022   Prob (F-statistic):          9.44e-115
Time:                        20:04:22   Log-Likelihood:                -1558.9
No. Observations:                 420   AIC:                             3126.
Df Residuals:                     416   BIC:                             3142.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    653.3426      0.486   1345.704      0.0

In [6]:
fm3 = """math ~ lunch*income + english"""

y3, X3 = dmatrices(fm3, data=df, return_type='dataframe')

mod3 = sm.OLS(y3, X3)
res3 = mod3.fit()

print(res3.summary())

string = res3.summary().as_latex()
txt_file = open(fig_dir / "37-final-mod3-summary.tex", 'w')
txt_file.write(string)
txt_file.close()

                            OLS Regression Results                            
Dep. Variable:                   math   R-squared:                       0.724
Model:                            OLS   Adj. R-squared:                  0.722
Method:                 Least Squares   F-statistic:                     272.5
Date:                Thu, 28 Jul 2022   Prob (F-statistic):          1.25e-114
Time:                        20:04:22   Log-Likelihood:                -1556.1
No. Observations:                 420   AIC:                             3122.
Df Residuals:                     415   BIC:                             3142.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept      652.3207      0.649   1004.445   

## Internal validation

In [7]:
table = pd.DataFrame({"Model 1": [rg.sse(X1.values, y1.values), rg.press(X1.values, y1.values)],
                      "Model 2": [rg.sse(X2.values, y2.values), rg.press(X2.values, y2.values)],
                      "Model 3": [rg.sse(X3.values, y3.values), rg.press(X3.values, y3.values)]},
                     index=['SSE', 'PRESS'])

savetable(table, "38-internal-validation-table.tex")

C:\Users\gfitz\OneDrive\Desktop\projects\ca-schools\src\functions.py:7: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  string = table.to_latex()


Each of the models have prediction error sum of squares reasonably close to SSE, which indicates that MSE is a good indicator of their predictive capabilities. 

## External validation

We will now split the data into a training and validation set using a 50-50 split.

In [8]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.5, random_state=40, shuffle=True)

In [9]:
train_df.head()

,students,teachers,calworks,lunch,computer,expenditure,income,english,math
221,3744.207143,200.052619,-0.567541,-7.815237,320.616667,-578.660959,-0.718921,-9.507368,653.099976
50,6306.207143,315.432624,23.699057,34.806764,482.616667,-295.715158,-1.254588,28.316905,633.700012
35,-361.792857,-25.387376,5.905657,39.728563,-126.383333,-40.215647,-3.984088,19.697215,628.200012
212,-2471.792857,-119.817376,-11.347342,-34.578637,-284.383333,-427.545236,-1.686588,-6.214015,655.500000
280,-1806.792857,-82.417375,-5.172242,-19.099738,-198.383333,472.217459,-1.153588,-14.916574,657.500000


In [10]:
test_df.head()

,students,teachers,calworks,lunch,computer,expenditure,income,english,math
71,358.207143,12.142631,8.012759,23.691164,-30.383333,-485.741525,-1.585989,-1.338962,631.099976
142,5787.207143,262.352637,-0.847041,27.206865,1029.616667,-246.496408,-2.646688,27.981845,647.299988
139,7708.207143,410.982612,23.980857,13.840162,964.616667,143.789725,-2.973588,-3.414475,643.099976
102,-2018.792857,-98.087377,-4.721442,11.360364,-159.383333,-608.363596,-4.033588,-12.325532,638.200012
228,-2483.792857,-118.857376,-4.970142,-24.705237,-283.383333,323.378592,-1.604588,-15.768155,655.200012


### Model 1

In [11]:
train_y1, train_X1 = dmatrices(fm1, data=train_df, return_type='dataframe')

train_mod1 = sm.OLS(train_y1, train_X1)
train_res1 = train_mod1.fit()

In [12]:
test_y1, test_X1 = dmatrices(fm1, data=test_df, return_type='dataframe')

test_mod1 = sm.OLS(test_y1, test_X1)
test_res1 = test_mod1.fit()

In [13]:
table = pd.DataFrame({'train_mod1_coef': train_res1.params,
                      'test_mod1_coef': test_res1.params,
                      'train_mod1_std_err': train_res1.bse,
                      'test_mod1_std_err': test_res1.bse})

savetable(table, "39-train-test-estimates-mod1.tex")

table

C:\Users\gfitz\OneDrive\Desktop\projects\ca-schools\src\functions.py:7: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  string = table.to_latex()


,train_mod1_coef,test_mod1_coef,train_mod1_std_err,test_mod1_std_err
Intercept,651.891579,651.542605,0.960207,0.982492
calworks,-0.188861,-0.356300,0.111898,0.125143
lunch,-0.380887,-0.277864,0.059565,0.060623
income,0.152318,0.616919,0.230191,0.226573
english,-0.158103,-0.181639,0.053425,0.054990
calworks:income,-0.040440,-0.039106,0.016160,0.016885


The standard error estimates are in very close agreement. The regression parameters are all within two standard deviations of each other, which is reasonably close.

We will now look at the mean squared prediction error (MSPR) for when the trained model is used to predict the test set. This will help us calibrate the predictive power of Model 1.

In [14]:
pred_errors = test_y1.values - train_res1.predict(test_X1).values.reshape(210, 1)

print("Model 1 MSPR: ", (np.matmul(pred_errors.T, pred_errors) / test_res1.nobs)[0,0])
print("Model 1 MSE: ", train_res1.mse_resid)

Model 1 MSPR:  103.4783893615233
Model 1 MSE:  93.21804607828616


The MSPR for Model 1 is moderately close to MSE, which suggests that MSE is a reasonably valid indicator of the models predictive power.

### Model 2

In [15]:
train_y2, train_X2 = dmatrices(fm2, data=train_df, return_type='dataframe')

train_mod2 = sm.OLS(train_y2, train_X2)
train_res2 = train_mod2.fit()

In [16]:
test_y2, test_X2 = dmatrices(fm2, data=test_df, return_type='dataframe')

test_mod2 = sm.OLS(test_y2, test_X2)
test_res2 = test_mod2.fit()

In [17]:
table = pd.DataFrame({'train_mod2_coef': train_res2.params,
                      'test_mod2_coef': test_res2.params,
                      'train_mod2_std_err': train_res2.bse,
                      'test_mod2_std_err': test_res2.bse})

savetable(table, "40-train-test-estimates-mod2.tex")

table

C:\Users\gfitz\OneDrive\Desktop\projects\ca-schools\src\functions.py:7: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  string = table.to_latex()


,train_mod2_coef,test_mod2_coef,train_mod2_std_err,test_mod2_std_err
Intercept,653.620324,653.171653,0.674512,0.699460
lunch,-0.369942,-0.361408,0.045050,0.046174
income,0.642720,0.972691,0.118377,0.161748
english,-0.145489,-0.123443,0.052549,0.050404


The regression estimates and standard error estimates are in very close agreement except for the parameter lunch, which is still not unreasonably close. This suggests reasonable stability of the regression estimates.

We will now look at the mean squared prediction error (MSPR) for when the trained model is used to predict the test set.

In [18]:
pred_errors = test_y2.values - train_res2.predict(test_X2).values.reshape(210, 1)

print("Model 2 MSPR: ", (np.matmul(pred_errors.T, pred_errors) / test_res2.nobs)[0,0])
print("Model 2 MSE: ", train_res2.mse_resid)

Model 2 MSPR:  104.17321282808439
Model 2 MSE:  95.26577780062405


The MSPR for Model 2 is moderately close to MSE, which suggests that MSE is a reasonably valid indicator of the model's predictive power.

### Model 3

In [19]:
train_y3, train_X3 = dmatrices(fm3, data=train_df, return_type='dataframe')

train_mod3 = sm.OLS(train_y3, train_X3)
train_res3 = train_mod3.fit()

In [20]:
test_y3, test_X3 = dmatrices(fm3, data=test_df, return_type='dataframe')

test_mod3 = sm.OLS(test_y3, test_X3)
test_res3 = test_mod3.fit()

In [21]:
table = pd.DataFrame({'train_mod1_coef': train_res3.params,
                      'test_mod1_coef': test_res3.params,
                      'train_mod1_std_err': train_res3.bse,
                      'test_mod1_std_err': test_res3.bse})

savetable(table, "41-train-test-estimates-mod3.tex")

table

C:\Users\gfitz\OneDrive\Desktop\projects\ca-schools\src\functions.py:7: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  string = table.to_latex()


,train_mod1_coef,test_mod1_coef,train_mod1_std_err,test_mod1_std_err
Intercept,652.376792,652.117887,0.905688,0.936975
lunch,-0.423147,-0.395220,0.051771,0.050178
income,0.288637,0.718136,0.209704,0.221054
lunch:income,-0.008909,-0.008309,0.004371,0.004943
english,-0.142309,-0.131896,0.052175,0.050434


The parameter and standard error estimates are in close agreement except for lunch, which is still not unreasonably close.

We will now look at the mean squared prediction error (MSPR) for when the trained model is used to predict the test set.

In [22]:
pred_errors = test_y3.values - train_res3.predict(test_X3).values.reshape(210, 1)

print("Model 3 MSPR: ", (np.matmul(pred_errors.T, pred_errors) / test_res3.nobs)[0,0])
print("Model 3 MSE: ", train_res3.mse_resid)

Model 3 MSPR:  103.80235333205607
Model 3 MSE:  93.82872536576107


The MSPR for Model 3 is moderately close to MSE, which suggests that MSE is a reasonably valid indicator of the models predictive power.

## Final model: $Y = X_0 + X_4 + X_7 + X_8 + \epsilon$

We recommend Model 2 for the final model because it best fulfills the criteria of an explanatory model. Its regression estimates, for income in particular, are more stable. Althought it has slightly less predictive capabilities, the difference is not substantial and it achieves this with the fewest parameters, which means it is the most interpretable.

Our model suggests that the variables that best explain differences in 5th grade math performance are all student demographic background variables: lunch, income, and english. These three variables explain 72% of the variation in standardized math scores. A 10 percentage point increase in the number of students on reduced-price lunch tends to be associated with a 3.8 point decrease on the average math score in the district. A 10,000 increase in the average family income for a district is associated with a 7.5 point increase on the average math score in the district. And a 10% percentage point increase in the number of English language learners is associated with a 1.3 point decrease on the average math score in the district.

In [23]:
print(res2.summary())

                            OLS Regression Results                            
Dep. Variable:                   math   R-squared:                       0.721
Model:                            OLS   Adj. R-squared:                  0.719
Method:                 Least Squares   F-statistic:                     357.5
Date:                Thu, 28 Jul 2022   Prob (F-statistic):          9.44e-115
Time:                        20:04:23   Log-Likelihood:                -1558.9
No. Observations:                 420   AIC:                             3126.
Df Residuals:                     416   BIC:                             3142.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    653.3426      0.486   1345.704      0.0